## **Developing the Front End**

In [ ]:
!pip install pymupdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 42.8 MB/s eta 0:00:00


In [ ]:
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 946.1 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.3 MB/s eta 0:00:00


In [ ]:
!wget -q -O - ipv4.icanhazip.com

34.9.47.81


In [ ]:
! streamlit run app.py & npx localtunnel --port 8501



⠙⠹
  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.9.47.81:8501

⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴Need to install the following packages:
localtunnel@2.0.2
Ok to proceed? (y) y

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇your url is: https://great-news-chew.loca.lt
────────────────────────── Traceback (most recent call last) ───────────────────────────
  /usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/exec_code.py:  
  128 in exec_func_with_error_handling                                                  
                                                                                        
  /usr/local/lib/python3.11/dist-packages/streamlit/runtime/scriptrunner/script_runner  
  .py:669 in code_to_exec                                                               
                                                                                        
  /content/app.py:14 in <module>    

# ***---------------------------------------------------------------------------------------------***

# **Setting up of VECTOR DATABASE**

In [ ]:
!pip install langchain \
            langchain-community \
            langchain-core \
            langchain-groq \
            faiss-cpu \
            pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.8/130.8 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 55.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.9 MB/s eta 0:00:00


In [ ]:
!pip install langchain-huggingface

In [ ]:
from langchain_community.document_loaders import PDFPlumberLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

### **Step 1: Upload & Load raw PDF(s)**


In [ ]:
pdfs_directory = 'pdfs/'

In [ ]:
def upload_pdf(file):
    with open(pdfs_directory + file.name, "wb") as f:
        f.write(file.getbuffer())

In [ ]:
def load_pdf(file_path):
    loader = PDFPlumberLoader(file_path)
    documents = loader.load()
    return documents

In [ ]:
file_path = '/content/universal_declaration_of_human_rights (1).pdf'
documents = load_pdf(file_path)

### **Step 2: Create Chunks**

In [ ]:
def create_chunks(documents):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size = 1000,
        chunk_overlap = 200,
        add_start_index = True
    )
    text_chunks = text_splitter.split_documents(documents)
    return text_chunks

text_chunks = create_chunks(documents)

In [ ]:
print("Chunks count: ", len(text_chunks))

Chunks count:  16


In [ ]:
print("PDF pages: ",len(documents))

PDF pages:  8


### **Step 3: Setup Embeddings Model (Use HuggingFace version --- *all-MiniLM-L6-v2*)**

In [ ]:
huggingface_model_name="all-MiniLM-L6-v2"

In [ ]:
def get_embedding_model(huggingface_model_name):
    embeddings = HuggingFaceEmbeddings(model=huggingface_model_name)
    return embeddings

### **Step 4: Index Documents Store embeddings in FAISS (vector store)**

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

faiss_db = FAISS.from_documents(text_chunks, embedding_model)
faiss_db.save_local("vectorstore/db_faiss")

/tmp/ipython-input-18-677494729.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

# **-----------------------------------------------------------------------------------------------**

# **Implementing and Initializing the RAG Pipeline**

In [ ]:
from langchain_groq import ChatGroq
from vectorstore import db_faiss
from langchain_core.prompts import ChatPromptTemplate

###**Step1: Setup LLM (Use DeepSeek R1 with Groq)**

In [ ]:
import os
os.environ["GROQ_API_KEY"] = ##"API KEY"

In [ ]:
from langchain_groq import ChatGroq

llm_model = ChatGroq(
    model_name="deepseek-r1-distill-llama-70b",
    api_key=os.environ["GROQ_API_KEY"]
)

###**Step2: Retrieve Docs**

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/RAG with Hugging Face")

from vectorstore import db_faiss

In [ ]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



In [ ]:
faiss_db = FAISS.load_local(
    "vectorstore/db_faiss",
    embedding_model,
    allow_dangerous_deserialization=True
)

In [ ]:
def retrieve_docs(faiss_db, query):
    return faiss_db.similarity_search(query)

In [ ]:
def retrieve_docs(faiss_db, query):
    return faiss_db.similarity_search(query)

def get_context(documents):
    context = "\n\n".join([doc.page_content for doc in documents])
    return context

###**Step3: Answer Question**

In [ ]:
custom_prompt_template = """
Use the pieces of information provided in the context to answer user's question.
If you dont know the answer, just say that you dont know, dont try to make up an answer.
Dont provide anything out of the given context
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
def answer_query(documents, model, query):
    context = get_context(documents)
    prompt = ChatPromptTemplate.from_template(custom_prompt_template)
    chain = prompt | model
    return chain.invoke({"question": query, "context": context})

In [ ]:
question="If a government forbids the right to assemble peacefully which articles are violated and why?"
retrieved_docs=retrieve_docs(faiss_db, question)
print("AI Lawyer: ",answer_query(documents=retrieved_docs, model=llm_model, query=question))

AI Lawyer:  content="<think>\nOkay, I need to figure out which articles are violated if a government forbids the right to assemble peacefully. Let me start by looking through each article one by one.\n\nFirst, Article 19 talks about freedom of opinion and expression, including imparting information. That doesn't seem directly related to assembly, so probably not this one.\n\nNext is Article 20. Section 1 explicitly states the right to peaceful assembly and association. So if a government forbids peaceful assembly, Article 20 is definitely violated.\n\nLooking at the other articles: Article 21 is about protecting moral and material interests, so that's about intellectual property, not relevant here. Article 28 is about social and international order, which is broader but doesn't specifically address assembly.\n\nArticle 29 mentions limitations on rights for public order and general welfare, but it's more about justifying restrictions rather than granting the right itself. So the violati

In [ ]:
print(type(faiss_db))


<class 'langchain_community.vectorstores.faiss.FAISS'>
